In [12]:
import os

In [2]:
%pwd

'/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject/research'

In [3]:
%cd /home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject/

/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject


In [4]:
%pwd

'/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject'

In [5]:
from dataclasses import dataclass
from  pathlib import Path 
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path 
    updated_base_model_path: Path 
    training_data: Path 
    params_epochs: int 
    params_batch_size: int 
    params_is_augmentation: bool
    params_image_size: list

In [6]:
@dataclass(frozen = True)
class CallBacksPrepConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml,create_directories

In [8]:
import tensorflow as tf

2023-10-09 18:18:20.532191: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 18:18:21.005441: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 18:18:21.007942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 18:18:22.487181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [30]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    def GetCallBacksPrepConfig(self)->CallBacksPrepConfig:
        config = self.config.callbacks_prep
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])
        callback_prep_config = CallBacksPrepConfig(root_dir=Path(config.root_dir), tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir), checkpoint_model_filepath=Path(config.checkpoint_model_filepath))
        return callback_prep_config
    def GetModelTrainingConfig(self)->ModelTrainingConfig:
        model_training = self.config.model_training
        base_model_prep = self.config.base_model_prep
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(model_training.root_dir)])
        model_training_config = ModelTrainingConfig(root_dir=Path(model_training.root_dir), trained_model_path=Path(model_training.trained_model_path),updated_base_model_path=Path(base_model_prep.updated_base_model_path),training_data=Path(training_data), params_epochs=params.EPOCHS, params_batch_size=params.BATCH_SIZE, params_is_augmentation=params.AUGMENTATION, params_image_size=params.IMAGE_SIZE)
        return model_training_config



In [31]:
import time

In [32]:
class CallBacksPrep:
    def __init__(self, config: CallBacksPrepConfig):
        self.config = config
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [33]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [39]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                        **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    def train(self, callbacks_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks_list
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [40]:
try:
    config = ConfigurationManager()
    callbacks_prep_config = config.GetCallBacksPrepConfig()
    callbacks_prep = CallBacksPrep(config=callbacks_prep_config)
    callbacks_list = callbacks_prep.get_tb_ckpt_callbacks()
    model_training_config = config.GetModelTrainingConfig()
    model_training = ModelTraining(config=model_training_config)
    model_training.get_base_model()
    model_training.train_valid_generator()
    model_training.train(callbacks_list=callbacks_list)
except Exception as e:
    raise e

[2023-10-09 19:32:41,693: INFO: common yaml file: config/config.yaml loaded succuessfully]
[2023-10-09 19:32:41,695: INFO: common yaml file: params.yaml loaded succuessfully]
[2023-10-09 19:32:41,697: INFO: common directory at artifacts created succuessfully]
[2023-10-09 19:32:41,699: INFO: common directory at artifacts/prepare_callbacks/checkpoint_dir created succuessfully]
[2023-10-09 19:32:41,701: INFO: common directory at artifacts/prepare_callbacks/tensorboard_log_dir created succuessfully]
[2023-10-09 19:32:41,703: INFO: common directory at artifacts/training created succuessfully]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10


2023-10-09 19:32:44.401905: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2023-10-09 19:32:44.416900: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-09 19:32:45.038224: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-09 19:32:45.414804: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2023-10-09 19:32:46.852962: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


9/9 [==============================] - ETA: 0s - loss: 13.6200 - accuracy: 0.4893 

/home/user/anaconda3/envs/chicken/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9/9 [==============================] - 173s 19s/step - loss: 13.6200 - accuracy: 0.4893 - val_loss: 9.0688 - val_accuracy: 0.3906
Epoch 2/10
9/9 [==============================] - 202s 23s/step - loss: 15.9462 - accuracy: 0.4643 - val_loss: 8.2863 - val_accuracy: 0.6094
Epoch 3/10
9/9 [==============================] - 179s 21s/step - loss: 8.5987 - accuracy: 0.5714 - val_loss: 6.0462 - val_accuracy: 0.4688
Epoch 4/10
9/9 [==============================] - 282s 34s/step - loss: 3.3262 - accuracy: 0.7179 - val_loss: 10.1136 - val_accuracy: 0.6094
Epoch 5/10
9/9 [==============================] - 365s 42s/step - loss: 4.9284 - accuracy: 0.7036 - val_loss: 0.8933 - val_accuracy: 0.8594
Epoch 6/10
9/9 [==============================] - 335s 38s/step - loss: 0.8292 - accuracy: 0.8929 - val_loss: 0.7089 - val_accuracy: 0.9219
Epoch 7/10
9/9 [==============================] - 313s 35s/step - loss: 2.5037 - accuracy: 0.7643 - val_loss: 1.2065 - val_accuracy: 0.8125
Epoch 8/10
9/9 [============